In [3]:
import os 
import json
from ai2thor.controller import Controller
import time
import pickle
import copy

In [4]:
def run(floorPlan, object_poses, actions, init_action, object_toggles, dirty_and_empty , turk_annotations, controller, task_type):
    
    controller.reset(scene_name =  floorPlan)
    
    if len(object_toggles) > 0:
            controller.step((dict(action='SetObjectToggles', objectToggles=object_toggles)))

    if dirty_and_empty:
            controller.step(dict(action='SetStateOfAllObjects',
                               StateChange="CanBeDirty",
                               forceAction=True))
            controller.step(dict(action='SetStateOfAllObjects',
                               StateChange="CanBeFilled",
                               forceAction=False))

    tmp = dict( action ='SetObjectPoses', objectPoses=object_poses)
 
    controller.step(tmp)
    
    # frame  = controller.last_event.frame
    controller.step(init_action)
    items = []
    subgoals = 0
    for action in actions:
        subgoals = max(action['high_idx'], subgoals)
    subgoals += 1
    
    for action in actions:
        precondition = controller.last_event.frame
        # print(action['api_action'])
        instance = controller.last_event.instance_segmentation_frame
        action['api_action'][ 'renderObjectImage'] = True
        controller.step(action['api_action'] )
        print(controller.last_event.instance_segmentation_frame)
        # print(controller.last_event.metadata['lastActionSuccess'])
        posteffect = controller.last_event.frame
        item = {
            'precondition': precondition, 
            'posteffect': posteffect, 
            'action' : action['api_action'], 
            'subgoals': subgoals, 
            'current_goal': action['high_idx'], 
            'currentprogress': float(action['high_idx'])/float(subgoals), 
            'low_descs': [ a['high_descs'][action['high_idx']] for a in  turk_annotations] , 
            'task_descs': [ a['task_desc'] for a in turk_annotations ],
            'task_type': task_type
        }
        items.append(item)
        # cur_high_idx = 0
        
        # for a in turk_annotations:
        #     try:
        #         item['low_descs'] = None
        #         item['task_desc'] = None
        #         item['low_descs'] = a['high_descs'][action['high_idx']]
        #         item['task_desc'] = a['task_desc']
               
        #         if item['low_descs'] and item['task_desc']:
        #             items.append(copy.deepcopy(item))
                
        #     except:
        #         pass

    return items



In [ ]:
def generate_high_plan_argument(data):
    high_plan_argument = {0 : {}, 1: {}}
    for idx, item in enumerate(data['plan']['high_pddl']):
                if len(item['discrete_action']['args']) > 0 :
                    for i in range(len(item['discrete_action']['args'])):
                        if item['discrete_action']['args'][i].endswith('bottle'):
                            item['discrete_action']['args'][i] = 'bottle'

                
                if item['discrete_action']['action'] == 'NoOp':
                    break
                if item['discrete_action']['action'] not in high_plan:
                    high_plan[item['discrete_action']['action']] = []
                try:
                    if len(item['discrete_action']['args']) > 0:
                            for i in range(len(item['discrete_action']['args'])):
                                if  item['discrete_action']['args'][i] not in high_plan_argument:
                                    if not item['discrete_action']['args'][i] == '':
                                        high_plan_argument[ item['discrete_action']['args'][i] ] = []
    return high_plan_argument



In [ ]:
def generate_high_plan(data):
    high_plan = {}
    for idx, item in enumerate(data['plan']['high_pddl']):
        if item['discrete_action']['action'] == 'NoOp':
            break
        if item['discrete_action']['action'] not in high_plan:
            high_plan[item['discrete_action']['action']] = []
            high_plan[item['discrete_action']['action']].append()
    

In [ ]:
def generate_plan(data):
    


In [7]:
split = 'train'
tasks = os.listdir('data/full_2.1.0/' + split)
cnt = 0
# controller = Controller()
# controller.start(player_screen_height=300,
#                  player_screen_width=300)
from tqdm import tqdm
table = []
jstring = {}
high_plan = {}
task_init_actions = {}
high_plan_argument = {0 : {}, 1: {}}
multi_argument = []
task_actions  = {}

for task in tqdm(tasks):
    task_path =  os.path.join('data/full_2.1.0/' + split, task)
    trials = os.listdir(task_path)
    
    for trial in trials:
        path = os.path.join('data/full_2.1.0/' + split, task, trial, 'traj_data.json')
        
        with open(path) as f:
            data = json.load(f)

        floorPlan = data['scene']['floor_plan']
        object_poses = data['scene']['object_poses']
        actions = data['plan']['low_actions']
        init_action = data['scene']['init_action']
        object_toggles  = data['scene']['object_toggles']
        dirty_and_empty = data['scene']['dirty_and_empty']
        turk_annotations =  data['turk_annotations']['anns']
        pddl = data['plan']['high_pddl']

        for idx, item in enumerate(data['plan']['high_pddl'])
        for action in actions:
            api = action['api_action']

        # if dirty_and_empty:
        #     print(dirty_and_empty)
        # continue

        for idx, item in enumerate(data['plan']['high_pddl']):
                if len(item['discrete_action']['args']) > 0 :
                    for i in range(len(item['discrete_action']['args'])):
                        if item['discrete_action']['args'][i].endswith('bottle'):
                            item['discrete_action']['args'][i] = 'bottle'

                
                if item['discrete_action']['action'] == 'NoOp':
                    break
                if item['discrete_action']['action'] not in high_plan:
                    high_plan[item['discrete_action']['action']] = []
                try:
                    if len(item['discrete_action']['args']) > 0:
                            for i in range(len(item['discrete_action']['args'])):
                                if  item['discrete_action']['args'][i] not in high_plan_argument:
                                    if not item['discrete_action']['args'][i] == '':
                                        high_plan_argument[ item['discrete_action']['args'][i] ] = []
                except:
                    # print(item['discrete_action'])
                    pass
               

                for annotation in turk_annotations:
                    try:
                        
                        high_plan[item['discrete_action']['action']].append(annotation['high_descs'][idx])
                        if item['discrete_action']['action'] == 'GotoLocation':
                            continue
                        if len(item['discrete_action']['args']) > 0 :
                            for i in range(len(item['discrete_action']['args'])):
                                if item['discrete_action']['args'][i] == '':
                                    continue
                                # if item['discrete_action']['args'][-1] == 'winebottle':
                                #     print(item['discrete_action'])
                                #     print(annotation['high_descs'][idx])

                                high_plan_argument[i][item['discrete_action']['args'][i]].append(annotation['high_descs'][idx])
                        # if len(item['discrete_action']['args']) > 1:
                        #     # print(item['discrete_action'])
                        #     multi_argument.append(item['discrete_action'])
                    except:
                        # print(idx, item['discrete_action']['action'], annotation['high_descs'])
                        # for a in data['plan']['high_pddl']:
                        #     print(a['discrete_action']['action'])
                        pass
        subgoals = 0
        for action in actions:
            subgoals = max(action['high_idx'], subgoals)

        current_goal = 0
        actions_so_far = []
        previous_task = data['plan']['high_pddl'][current_goal]['discrete_action']['action']
        previous_goal = current_goal
        for action in actions:
            previous_goal = current_goal
            
            current_goal = action['high_idx']
            current_task = data['plan']['high_pddl'][current_goal]['discrete_action']['action']
            # if previous_task is not current_task:
            #     entry = {}
            #     entry['init_action'] = init_action
            #     entry['pre_action'] = actions_so_far
            #     entry['poses'] = object_poses
            #     entry['floorPlan'] = floorPlan
            #     entry['object_toggles'] = object_toggles
            #     entry['dirty_and_empty'] = dirty_and_empty
            #     entry['high_descs'] = [ a['high_descs'][action['high_idx']] for a in  turk_annotations]
            #     entry['task_descs'] = [ a['task_desc'] for a in  turk_annotations ]
                # entry['traj'] = traj
                

            previous_task = current_task
            actions_so_far.append(action)

        if  data['task_type'] not in jstring:
            jstring[data['task_type'] ] = []
            jstring[data['task_type'] ].append(turk_annotations)
        else:
            jstring[data['task_type'] ].append(turk_annotations)
        
        # try:
        #     eposide = run(floorPlan, object_poses, actions, init_action, object_toggles, dirty_and_empty, turk_annotations , controller, data['task_type'])
        #     table.append(eposide)
        # except:
        #     pass
        
    continue
    if len(table) > 500 or task == tasks[-1]:
        with open('dataset_'+split+'_'+str(cnt)+'.pickle', 'wb') as handle:
            pickle.dump(table, handle, protocol=pickle.HIGHEST_PROTOCOL)
        cnt += 1
        table = []
    
        # for action in actions:
        #         print(action['api_action'])
       
# controller.stop()

100%|██████████| 2435/2435 [00:08<00:00, 283.19it/s]


In [4]:
data.keys()

dict_keys(['images', 'pddl_params', 'plan', 'scene', 'task_id', 'task_type', 'turk_annotations'])

In [5]:
data['plan']['high_pddl']
for item in data['plan']['high_pddl']:
    print(item['discrete_action']['action'])

GotoLocation
PickupObject
GotoLocation
CleanObject
GotoLocation
PutObject
NoOp


In [6]:

data['turk_annotations']

{'anns': [{'assignment_id': 'A1ELPYAFO7MANS_3BDCF01OG0L7YJZHHESWAP3ZNI7LY7',
   'high_descs': ['Walk ahead to the door, then turn left and take a step, then turn left and face the counter.',
    'Pick up the dirty plate on the counter.',
    'Walk left around the counter, and straight to the sink.',
    'Clean the plate in the sink.',
    'Turn left and walk to the fridge.',
    'Place the plate on the top shelf of the fridge.'],
   'task_desc': 'Place a rinsed plate in the fridge.',
   'votes': [1, 1]},
  {'assignment_id': 'A20FCMWP43CVIU_3FPRZHYEP1Y9ELEN6X1C745O5FD3V1',
   'high_descs': ['walk to face bar counter top',
    'pick up plate from counter ',
    'walk to face sink',
    'wash and remove plate from sink',
    'walk to face fridge',
    'put plate inside fridge'],
   'task_desc': 'put washed plate inside fridge',
   'votes': [1, 1]},
  {'assignment_id': 'A3HL2LL0LEPZT8_3I3WADAZ9TVH2GSUJ43RR990NHMO5N',
   'high_descs': ['Go forward and right to the door, turn left, go forwar

In [7]:
from collections import defaultdict, Counter
lens = {}
for k,v in jstring.items():
    lens[k] = []
    for p in v:
       for item in p:
           lens[k].append(len(item['high_descs']))

lens.keys()   


dict_keys(['look_at_obj_in_light', 'pick_cool_then_place_in_recep', 'pick_heat_then_place_in_recep', 'pick_and_place_with_movable_recep', 'pick_and_place_simple', 'pick_clean_then_place_in_recep', 'pick_two_obj_and_place'])

In [8]:
print(Counter(lens['look_at_obj_in_light']))
print(Counter(lens['pick_cool_then_place_in_recep']))
print(Counter(lens['pick_heat_then_place_in_recep']))
print(Counter(lens['pick_and_place_with_movable_recep']))
print(Counter(lens['pick_clean_then_place_in_recep']))
print(Counter(lens['pick_and_place_simple']))
print(Counter(lens['pick_two_obj_and_place']))


Counter({4: 94})
Counter({6: 87, 12: 27, 11: 12})
Counter({6: 60, 12: 41, 9: 3, 10: 3})
Counter({7: 52, 6: 18, 11: 14, 13: 12, 10: 9, 12: 5, 4: 5})
Counter({6: 99, 12: 10, 11: 3})
Counter({4: 142})
Counter({8: 118, 16: 6})


In [9]:
import torch
vocab = torch.load('pp.vocab')

In [10]:
# vocab['word'].word2index('')

In [11]:
high_plan.keys()

dict_keys(['GotoLocation', 'PickupObject', 'ToggleObject', 'CoolObject', 'PutObject', 'SliceObject', 'HeatObject', 'CleanObject'])

In [12]:
import json
with open('dataset_high_plan_' + split + '.json', 'w') as fp:
    json.dump(high_plan, fp)

In [17]:
data['plan']['high_pddl'][0]['discrete_action']['args']

['countertop']

In [14]:
len(list(high_plan_argument.keys()))

81

In [10]:
with open('dataset_high_plan_argument_' + split + '.json', 'w') as fp:
    json.dump(high_plan_argument, fp)

In [8]:
action_set = set()
for item in multi_argument:
    action_set.add(item['action'])
print(action_set)


set()
